In [13]:
from PIL import Image, ImageOps
import os

# Simulate retrieving files from a directory
# For this example, the images need to be in a specific folder.
# Assuming we have individual image files for each character in a directory.
# The folder structure and filenames need to be set as per the user's environment.
# Replace with the actual directory containing images
# 
#image_folder = "./123/Chinese_User_3_full"
image_folder = "/Users/fanyuxin/Desktop/ShanghiTech/2024_autumn/计算机视觉/Project/zcb_input"

# Target phrase to render
phrase = "大"

# Function to find image paths for each character in the phrase


def find_images_for_phrase(phrase, folder):
    images = []
    for char in phrase:
        char_filename = f"{char}.png"
        char_path = os.path.join(folder, char_filename)
        if os.path.exists(char_path):
            images.append(char_path)
        else:
            print(f"Image for '{char}' not found in {folder}.")
    return images


# Find images
image_paths = find_images_for_phrase(phrase, image_folder)

# Combine images into a single row


def combine_images_horizontally(image_paths):
    images = [Image.open(img_path) for img_path in image_paths]
    widths, heights = zip(*(img.size for img in images))

    total_width = sum(widths)
    max_height = max(heights)

    combined_image = Image.new("RGB", (total_width, max_height), "white")
    x_offset = 0
    for img in images:
        combined_image.paste(img, (x_offset, 0))
        x_offset += img.size[0]

    return combined_image


# Generate the combined image
if image_paths:
    combined_image = combine_images_horizontally(image_paths)
    output_path = "./combined_phrase_image.png"
    combined_image.save(output_path)
    output_path
else:
    "No images were found for the given phrase."

Image for '大' not found in /Users/fanyuxin/Desktop/ShanghiTech/2024_autumn/计算机视觉/Project/zcb_input.


In [14]:
from PIL import Image, ImageOps
import os
import numpy as np

# Directory containing individual character images
# image_folder = "./Chinese_User"

# Target phrase to render
phrase = "人才力气大人才力气大人才力气大人才力气大人才力气大"
# phrase = "全面建设社会主义现代化国家实现新时代新征程各项目标任务关键在党持之以恒推进全面从严治党深入推进新时代党的建设新的伟大工程就要做到坚持和加强党的全面领导与坚持全面从严治党有机统一坚持思想建党与制度治党同向发力坚持人民监督与自我革命相互结合坚持全面推进与突出重点相辅相成坚持着力治标与注重治本同频共振"

# Transformation matrix: (scale, rotation_angle, translate_x, translate_y)
# Replace this with the actual transformation matrix
# transform_matrix = [
#     # scale, rotation_angle, translate_x, translate_y
#     [1.0, 0, 0, 15],
#     [0.8, 15, 10, 20],
#     [1.2, -30, 50, 50],
#     [1.0, 10, -60, 100],
#     [1.0, 10, -60, 100],
# ]

# transform_matrix=np.tile(transform_matrix,(20,1)).tolist()
transform_matrix = np.zeros((len(phrase), 4))
transform_matrix[:,0] = np.random.normal(1,0.2,len(phrase))
transform_matrix[:,1] = np.random.normal(0,15,len(phrase))
transform_matrix[:,2] = np.random.normal(0,15,len(phrase))
transform_matrix[:,3] = np.random.normal(0,50,len(phrase))
transform_matrix = transform_matrix.tolist()

# Ensure the transformation matrix matches the phrase length
assert len(transform_matrix) == len(
    phrase), "Transformation matrix must match the phrase length."

# Function to find image paths for each character in the phrase

def find_images_for_phrase(phrase, folder):
    images = []
    for char in phrase:
        char_filename = f"{char}.jpg"
        char_path = os.path.join(folder, char_filename)
        if os.path.exists(char_path):
            images.append(char_path)
        else:
            print(f"Image for '{char}' not found in {folder}. Using placeholder.")
            # Create a placeholder blank image if not found
            # Adjust size as needed
            blank_image = Image.new("RGB", (50, 50), "white")
            images.append(blank_image)
    return images

# Combine images with transformations


def combine_images_with_transformations(images, transform_matrix):
    total_width = 0
    max_height = 0

    transformed_images = []
    x_offset = 0  # Starting x-offset
    for i, img in enumerate(images):
        if isinstance(img, str):
            img = Image.open(img)
        scale, rotation, translate_x, translate_y = transform_matrix[i]
        translate_x = int(translate_x)
        translate_y = int(translate_y)

        # Apply scaling
        width, height = img.size
        new_size = (int(width * scale), int(height * scale))
        img = img.resize(new_size, Image.Resampling.LANCZOS)

        # Apply rotation
        img = img.rotate(rotation, expand=True,fillcolor="white")

        # Update offsets
        x_offset += translate_x
        y_offset = translate_y

        # Keep track of max height for canvas sizing
        max_height = max(max_height, img.size[1] + abs(y_offset))

        # Append transformed image with its position
        transformed_images.append((img, x_offset, y_offset))
        x_offset += img.size[0]  # Add the width of the current image

    # Create a blank canvas for the combined image
    total_width = x_offset
    canvas = Image.new("RGB", (total_width, max_height), "white")

    # Place transformed images onto the canvas
    current_x = 0
    for img, x_offset, y_offset in transformed_images:
        canvas.paste(img, (x_offset, y_offset))
        current_x += img.size[0] + translate_x

    return canvas


# Load images for the characters
image_paths = find_images_for_phrase(phrase, image_folder)

# Generate the combined image
# if image_paths:
#     combined_image = combine_images_with_transformations(
#         image_paths, transform_matrix)
#     output_path = "./combined_transformed_phrase_image.png"
#     combined_image.save(output_path)
#     print(f"Output saved to {output_path}")
# else:
#     print("No images were found for the given phrase.")

In [15]:
def combine_images_with_line_breaks(images, transform_matrix, max_line_width):
    x_offset = 0  # Starting x-offset
    y_offset = 0  # Starting y-offset
    line_height = 0  # Height of the current line
    max_width = 0
    total_height = 0

    transformed_images = []
    for i, img in enumerate(images):
        if isinstance(img, str):
            img = Image.open(img)
        scale, rotation, translate_x, translate_y = transform_matrix[i]
        translate_x = int(translate_x)
        translate_y = int(translate_y)

        # Apply scaling
        width, height = img.size
        new_size = (int(width * scale), int(height * scale))
        img = img.resize(new_size, Image.Resampling.LANCZOS)

        # Apply rotation
        img = img.rotate(rotation, expand=True, fillcolor="white")

        # Check if we need to wrap to the next line
        if x_offset + img.size[0] > max_line_width:
            x_offset = 0
            y_offset += line_height  # Move down by the height of the tallest image in the line
            line_height = 0  # Reset line height for the new line

        # Update line height
        line_height = max(line_height, img.size[1])

        # Append transformed image with its position
        transformed_images.append((img, x_offset, y_offset))

        # Update offsets for the next image
        x_offset += img.size[0] + translate_x
        max_width = max(max_width, x_offset)
        total_height = max(total_height, y_offset + line_height)

    # Create a blank canvas for the combined image
    canvas = Image.new("RGB", (max_line_width, total_height), "white")

    # Place transformed images onto the canvas
    for img, x_pos, y_pos in transformed_images:
        canvas.paste(img, (x_pos, y_pos))

    return canvas


# Specify the maximum line width for wrapping (e.g., 800 pixels)
max_line_width = 3000

# Generate the combined image with line breaks
if image_paths:
    try:
        combined_image = combine_images_with_line_breaks(
            image_paths, transform_matrix, max_line_width)
        output_path = "./combined_transformed_phrase_with_linebreaks.png"
        combined_image.save(output_path)
        print(f"Output saved to {output_path}")
    except Exception as e:
        print(f"Error during image generation: {e}")
else:
    print("No images were found for the given phrase.")

Output saved to ./combined_transformed_phrase_with_linebreaks.png


In [16]:
import numpy as np
from PIL import Image


def combine_images_with_pixel_blending(images, transform_matrix, max_line_width, line_spacing=50):
    x_offset = 0  # Starting x-offset
    y_offset = 0  # Starting y-offset
    line_height = 0  # Height of the current line
    max_width = 0
    total_height = 0

    transformed_images = []
    for i, img in enumerate(images):
        if isinstance(img, str):
            img = Image.open(img)
        scale, rotation, translate_x, translate_y = transform_matrix[i]
        translate_x = int(translate_x)
        translate_y = int(translate_y)

        # Apply scaling
        width, height = img.size
        new_size = (int(width * scale), int(height * scale))
        img = img.resize(new_size, Image.Resampling.LANCZOS)

        # Apply rotation
        img = img.rotate(rotation, expand=True, fillcolor="white")

        # Ensure image has an alpha channel for blending
        img = img.convert("RGBA")

        # Check if we need to wrap to the next line
        if x_offset + img.size[0] > max_line_width:
            x_offset = 0
            y_offset += line_height+line_spacing  # Move down by the height of the tallest image in the line
            line_height = 0  # Reset line height for the new line

        # Update line height
        line_height = max(line_height, img.size[1])

        # Append transformed image with its position
        transformed_images.append((img, x_offset, y_offset))

        # Update offsets for the next image
        x_offset += img.size[0] + translate_x
        max_width = max(max_width, x_offset)
        total_height = max(total_height, y_offset + line_height)

    # Create a blank RGBA canvas for the combined image
    canvas = Image.new("RGBA", (max_line_width, total_height),
                       (255, 255, 255, 255))  # White background

    # Convert canvas to numpy array for faster processing
    canvas_array = np.array(canvas)

    # Place transformed images onto the canvas with pixel-level blending using numpy
    for img, x_pos, y_pos in transformed_images:
        img_array = np.array(img)  # Convert image to numpy array
        h, w, _ = img_array.shape

        # Extract the region from the canvas where the current image will be placed
        canvas_region = canvas_array[y_pos:y_pos + h, x_pos:x_pos + w]

        # Calculate brightness for blending
        canvas_brightness = 0.299 * canvas_region[:, :, 0] + \
            0.587 * canvas_region[:, :, 1] + \
            0.114 * canvas_region[:, :, 2]
        img_brightness = 0.299 * img_array[:, :, 0] + \
            0.587 * img_array[:, :, 1] + \
            0.114 * img_array[:, :, 2]

        # Create a mask where the image pixel is darker than the canvas pixel
        mask = img_brightness < canvas_brightness

        # Blend the image into the canvas
        canvas_region[mask] = img_array[mask]

        # Update the canvas region
        canvas_array[y_pos:y_pos + h, x_pos:x_pos + w] = canvas_region

    # Convert numpy array back to PIL Image
    final_image = Image.fromarray(canvas_array, mode="RGBA")
    return final_image


# Specify the maximum line width for wrapping (e.g., 800 pixels)
max_line_width = 6000
linespacing = 50

# Generate the combined image with pixel blending
if image_paths:
    try:
        combined_image = combine_images_with_pixel_blending(
            image_paths, transform_matrix, max_line_width,line_spacing=linespacing)
        output_path = "./combined_transformed_phrase_with_pixel_blending.png"
        combined_image.save(output_path)
        print(f"Output saved to {output_path}")
    except Exception as e:
        print(f"Error during image generation: {e}")
else:
    print("No images were found for the given phrase.")

Output saved to ./combined_transformed_phrase_with_pixel_blending.png
